<a href="https://colab.research.google.com/github/nguyentuyetnhung/btap/blob/main/du_doan_menh_gia_tien_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Flatten, Dropout, BatchNormalization, InputLayer
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from google.colab import drive
import gradio as gr
from tensorflow.keras.preprocessing.image import img_to_array
from PIL import Image
import matplotlib.pyplot as plt
import cv2

# ================== 1. KẾT NỐI GOOGLE DRIVE ==================
drive.mount("/content/drive")

# Thư mục chứa dataset trong Drive
base_dir = "/content/drive/MyDrive/codetien"

# Đường dẫn lưu mô hình
model_dir = "/content/drive/MyDrive/dudoangiatien"
os.makedirs(model_dir, exist_ok=True)  # Tạo thư mục nếu chưa tồn tại

best_model_path = os.path.join(model_dir, "best_model.h5")
final_model_path = os.path.join(model_dir, "doangiatien.h5")
history_plot_path = os.path.join(model_dir, "training_history.png")
labels_path = os.path.join(model_dir, "class_labels.npy")

# ================== 2. KIỂM TRA XEM MÔ HÌNH ĐÃ ĐƯỢC TRAIN CHƯA ==================
def load_trained_model():
    """Kiểm tra và tải mô hình đã train nếu có"""
    if os.path.exists(best_model_path) and os.path.exists(labels_path):
        print("✅ Đã tìm thấy mô hình đã train, đang tải...")
        model = load_model(best_model_path)

        # Tải labels từ file
        class_indices = np.load(labels_path, allow_pickle=True).item()
        labels = {v: k for k, v in class_indices.items()}
        print("✅ Đã tải labels từ file")
        return model, labels, class_indices
    else:
        print("⚠️ Không tìm thấy mô hình đã train hoặc file labels, cần train mới")
        return None, None, None

# Thử tải mô hình đã train
model, labels, class_indices = load_trained_model()

# Nếu chưa có mô hình, tiến hành train
if model is None:
    # ================== 3. TIỀN XỬ LÝ DỮ LIỆU ==================
    img_size = (150, 150)
    batch_size = 32

    print("Kiểm tra cấu trúc thư mục:")
    for root, dirs, files in os.walk(base_dir):
        level = root.replace(base_dir, '').count(os.sep)
        indent = ' ' * 2 * level
        print(f"{indent}{os.path.basename(root)}/")
        subindent = ' ' * 2 * (level + 1)
        for file in files[:3]:
            print(f"{subindent}{file}")
        if len(files) > 3:
            print(f"{subindent}... và {len(files) - 3} file khác")

    # Tăng cường dữ liệu mạnh hơn
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=30,
        width_shift_range=0.3,
        height_shift_range=0.3,
        horizontal_flip=True,
        vertical_flip=True,
        zoom_range=0.3,
        shear_range=0.2,
        brightness_range=[0.7, 1.3],
        fill_mode='nearest',
        validation_split=0.2
    )

    val_datagen = ImageDataGenerator(
        rescale=1./255,
        validation_split=0.2
    )

    train_generator = train_datagen.flow_from_directory(
        base_dir,
        target_size=img_size,
        batch_size=batch_size,
        class_mode="categorical",
        shuffle=True,
        subset="training"
    )

    val_generator = val_datagen.flow_from_directory(
        base_dir,
        target_size=img_size,
        batch_size=batch_size,
        class_mode="categorical",
        shuffle=False,
        subset="validation"
    )

    # Lưu class indices để sử dụng sau
    class_indices = train_generator.class_indices
    np.save(labels_path, class_indices)
    labels = {v: k for k, v in class_indices.items()}

    # Kiểm tra số lượng ảnh trong mỗi lớp training
    class_counts = {}
    for class_name, idx in class_indices.items():
        count = np.sum(train_generator.classes == idx)
        class_counts[class_name] = count
        print(f"Lớp {class_name}: {count} ảnh")

    # Kiểm tra cân bằng dữ liệu
    min_count = min(class_counts.values())
    max_count = max(class_counts.values())
    if max_count > 1.5 * min_count:
        print("\n⚠️ Cảnh báo: Dữ liệu không cân bằng! Có thể ảnh hưởng đến chất lượng mô hình.")

    # ================== 4. XÂY DỰNG MÔ HÌNH ANN ==================
    model = Sequential([
        InputLayer(input_shape=(img_size[0], img_size[1], 3)), # Explicitly define input shape
        Flatten(),
        Dense(2048, activation="relu"),
        BatchNormalization(),
        Dropout(0.6),
        Dense(1024, activation="relu"),
        BatchNormalization(),
        Dropout(0.5),
        Dense(512, activation="relu"),
        BatchNormalization(),
        Dropout(0.4),
        Dense(256, activation="relu"),
        BatchNormalization(),
        Dropout(0.3),
        Dense(128, activation="relu"),
        BatchNormalization(),
        Dropout(0.2),
        Dense(len(class_indices), activation="softmax")
    ])

    # ================== 5. COMPILE ==================
    model.compile(optimizer=Adam(learning_rate=0.0003),
                  loss="categorical_crossentropy",
                  metrics=["accuracy"])

    model.summary()

    # Callbacks
    early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=7, min_lr=0.00001)
    checkpoint = ModelCheckpoint(
        best_model_path,
        monitor='val_accuracy',
        save_best_only=True,
        mode='max',
        verbose=1
    )

    # ================== 6. TRAIN ==================
    print("Bắt đầu training...")
    history = model.fit(
        train_generator,
        steps_per_epoch=max(1, train_generator.samples // batch_size),
        validation_data=val_generator,
        validation_steps=max(1, val_generator.samples // batch_size),
        epochs=100,
        callbacks=[early_stopping, reduce_lr, checkpoint],
        verbose=1
    )

    # ================== 7. LƯU MÔ HÌNH CUỐI CÙNG ==================
    model.save(final_model_path)
    print(f"✅ Đã train xong và lưu mô hình ANN tại: {final_model_path}")

    # Vẽ biểu đồ accuracy và loss
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Model Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Model Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.tight_layout()
    plt.savefig(history_plot_path)
    plt.show()

else:
    print("✅ Đã tải mô hình đã train thành công!")
    # Get img_size from model input shape
    img_size = model.input_shape[1:3]  # Assuming the input layer is the first layer

# ================== 8. TẠO GIAO DIỆN GRADIO ==================
# Hàm xử lý ảnh
def preprocess_image(img):
    # Chuyển đổi sang RGB nếu là ảnh RGBA
    if img.mode == 'RGBA':
        img = img.convert('RGB')

    # Resize và chuẩn hóa
    img_resized = img.resize(img_size)
    img_array = img_to_array(img_resized) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    return img_array, img_resized

# Hàm dự đoán
def predict_and_show(img):
    if img is None:
        return "Vui lòng tải lên một ảnh hợp lệ", None

    try:
        img_array, processed_img = preprocess_image(img)

        # Dự đoán
        preds = model.predict(img_array, verbose=0)

        # Ensure preds is an array of probabilities
        if preds.shape == (1,):
          preds = np.array([preds]) # Reshape to (1, 1) if it's a single value

        class_idx = np.argmax(preds[0])
        confidence = np.max(preds[0])
        label = labels[class_idx]

        # Tạo biểu đồ xác suất
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

        # Hiển thị ảnh
        ax1.imshow(processed_img)
        ax1.set_title('Ảnh đã tải lên')
        ax1.axis('off')

        # Hiển thị biểu đồ xác suất
        classes = list(labels.values())
        probabilities = preds[0]
        colors = ['#FF6B6B', '#4ECDC4', '#45B7D1']
        bars = ax2.bar(classes, probabilities, color=colors)
        ax2.set_ylabel('Xác suất')
        ax2.set_title('Xác suất dự đoán')
        ax2.set_ylim(0, 1)

        # Thêm giá trị trên mỗi cột
        for bar, prob in zip(bars, probabilities):
            height = bar.get_height()
            ax2.text(bar.get_x() + bar.get_width()/2., height + 0.01,
                    f'{prob:.2%}', ha='center', va='bottom', fontweight='bold')

        plt.tight_layout()

        result_text = f"💵 Mệnh giá: {label} nghìn đồng\n🎯 Độ tin cậy: {confidence*100:.2f}%"
        return result_text, fig

    except Exception as e:
        return f"Lỗi xử lý ảnh: {str(e)}", None

# CSS tùy chỉnh
custom_css = """
.gradio-container {
    background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
    font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
    max-width: 1200px !important;
    margin: 20px auto;
    border-radius: 20px;
    box-shadow: 0 15px 35px rgba(0,0,0,0.2);
    padding: 30px;
}
h1 {
    text-align: center;
    color: white;
    font-weight: 700;
    margin-bottom: 10px;
    font-size: 2.5em;
    text-shadow: 2px 2px 4px rgba(0,0,0,0.3);
}
h2 {
    text-align: center;
    color: #f8f9fa;
    font-weight: 400;
    margin-top: 0;
    font-size: 1.3em;
}
.upload-box {
    border: 3px dashed #a991f7 !important;
    border-radius: 15px;
    padding: 25px;
    background-color: rgba(255, 255, 255, 0.1);
    backdrop-filter: blur(10px);
    min-height: 250px;
}
.upload-box:hover {
    border-color: #8a6df0 !important;
    background-color: rgba(255, 255, 255, 0.15);
}
button {
    background: linear-gradient(to right, #ff6b6b, #ff8e8e) !important;
    color: white !important;
    font-weight: bold;
    border-radius: 25px !important;
    padding: 15px 35px !important;
    border: none !important;
    box-shadow: 0 5px 15px rgba(255, 107, 107, 0.4);
    transition: all 0.3s ease !important;
    margin: 20px auto;
    display: block;
    font-size: 1.1em;
}
button:hover {
    transform: translateY(-3px);
    box-shadow: 0 8px 20px rgba(255, 107, 107, 0.6);
}
.output-textbox textarea {
    font-size: 22px !important;
    font-weight: bold !important;
    color: #2c3e50 !important;
    background: linear-gradient(135deg, #ffffff 0%, #f8f9fa 100%) !important;
    border-radius: 15px;
    padding: 25px !important;
    border: 2px solid #e0e0e0 !important;
    text-align: center;
    box-shadow: 0 5px 15px rgba(0,0,0,0.1);
}
.footer {
    text-align: center;
    margin-top: 30px;
    color: rgba(255, 255, 255, 0.8);
    font-size: 16px;
    padding: 20px;
    background-color: rgba(0, 0, 0, 0.2);
    border-radius: 15px;
    backdrop-filter: blur(5px);
}
.thumbnail {
    display: flex;
    justify-content: center;
    gap: 20px;
    margin: 25px 0;
}
.thumbnail img {
    width: 100px;
    height: 50px;
    border-radius: 8px;
    border: 3px solid #a991f7;
    padding: 5px;
    background: white;
    box-shadow: 0 4px 8px rgba(0,0,0,0.2);
    transition: transform 0.3s ease;
}
.thumbnail img:hover {
    transform: scale(1.1);
}
.instructions {
    background: rgba(255, 255, 255, 0.1);
    padding: 15px;
    border-radius: 10px;
    margin: 15px 0;
    color: white;
}
"""

# Tạo giao diện
with gr.Blocks(css=custom_css, title="Nhận Diện Mệnh Giá Tiền") as demo:
    gr.Markdown("# 💵 ỨNG DỨNG NHẬN DIỆN MỆNH GIÁ TIỀN")
    gr.Markdown("### Tải lên ảnh tờ tiền 100, 200 hoặc 500 nghìn đồng để nhận diện")

    gr.Markdown("""
    <div class="instructions">
    <strong>📝 Hướng dẫn:</strong><br>
    1. Chọn ảnh tờ tiền cần nhận diện (100k, 200k hoặc 500k)<br>
    2. Nhấn nút "🔍 Nhận diện mệnh giá" để phân tích<br>
    3. Xem kết quả và biểu đồ xác suất dự đoán
    </div>
    """)

    with gr.Row():
        gr.Markdown(
            """
            <div class="thumbnail">
                <img src="https://static.vecteezy.com/system/resources/previews/024/761/402/non_2x/cash-100000-vnd-banknote-close-up-of-vietnamese-money-isolated-on-white-background-vector.jpg" alt="100k">
                <img src="https://static.vecteezy.com/system/resources/previews/024/761/406/non_2x/cash-200000-vnd-banknote-close-up-of-vietnamese-money-isolated-on-white-background-vector.jpg" alt="200k">
                <img src="https://static.vecteezy.com/system/resources/previews/024/761/407/non_2x/cash-500000-vnd-banknote-close-up-of-vietnamese-money-isolated-on-white-background-vector.jpg" alt="500k">
            </div>
            """,
            elem_id="thumbnail"
        )

    with gr.Row():
        with gr.Column(scale=1):
            img_input = gr.Image(type="pil", label="📷 Tải lên ảnh tờ tiền", elem_classes="upload-box", height=300)
            submit_btn = gr.Button("🔍 Nhận diện mệnh giá", size="lg")

        with gr.Column(scale=1):
            output_text = gr.Textbox(label="📌 Kết quả nhận diện", interactive=False, lines=3)
            plot_output = gr.Plot(label="📊 Biểu đồ dự đoán")

    submit_btn.click(fn=predict_and_show, inputs=img_input, outputs=[output_text, plot_output])

    gr.Markdown("---")
    gr.Markdown(
        """
        <div class="footer">
        <strong>Ứng dụng nhận diện mệnh giá tiền Việt Nam</strong><br>
        Sử dụng mô hình Artificial Neural Network (ANN) để phân tích và nhận diện<br>
        Hỗ trợ các mệnh giá: 100, 200 và 500 nghìn đồng
        </div>
        """,
        elem_id="footer"
    )

print("🚀 Đang khởi chạy ứng dụng...")
demo.launch(debug=True, share=True, height=800)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Đã tìm thấy mô hình đã train, đang tải...


✅ Đã tải labels từ file
✅ Đã tải mô hình đã train thành công!
🚀 Đang khởi chạy ứng dụng...
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://5f32ecc44b1802c3ee.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
